# Generation of the OTI Lib pre-computed data.

## Introduction

This file focuses on generating the pre-computed data that serves to the OTI library. Particularly this library focuses on computing the following elements:

- Partition Sets for all (defined) orders and 
- Direction and Exponent arrays for all possible orders and number of variables.

For this, we'll need to load a combinatorics library from scipy, partition from partitionsets and numpy.

In [2]:
from partitionsets import partition
import numpy as np
from scipy.special import comb

## Generating the partition sets.

Partition sets can be generated using the partition library, through the call ```partition.Partition(set)```. It is very robust and well made for Python usage. However, being so robust and well set for many different kind of problems might result in a performance drawback of the OTI library. As a result, we opt to pre-compute the sets in the array and use them later.

The partition of a set only depend on the number of elements of the set. Since this partition applies only to partition of each direction array, and those have a maximum number of elements given by the maximum order. As a result partition of sets will only be generated with respect to the ```maxorder``` variable.

For now, with respect to particular applications, this method is limiting the maximum order to 15, due to the fact that it is using integers of 16 bits. If the element type is uint32 or uint64, the maximum order will be 33 or 63 respectively. 

In [1]:
# Define the order to create the partition values:
maxorder = 5

# Create the elements array
# Holds up to 15 elements due to uint16...
els =np.array(list(range(0,maxorder)),dtype=np.uint16)


# Generate the partition sets.
parts = list(partition.Partition(els))

# Generate the associated values. Here is where it matters to have 16, 32 or 64 bits.
values = np.power(np.uint16(2),els)
print(els)
numels = len(parts)
print("Number of partitions is: ",numels)

# create the Array to store the values of the partition information:
A = np.zeros((len(parts),maxorder),dtype = np.uint16)

print("Generating matrix of ",numels," partitions.")
# Travel all partition sets and store them in memor
for k in range(len(parts)):#part in parts:
    
    part = parts[k]
    
    
    for j in range(len(part)):#subset in part:
        
        subset = part[j]
        
        for i in range(len(subset)):
            
            A[k,j] +=values[subset[i]]
        
        # end for
        
    # end for 
    
# end for 

# Save the array:
filename = "data/part_n"+str(maxorder)
print("Saving file << ",filename,".npy >> into folder: data/ \n")
#np.save(filename,A)
print("============= Finished ============")

NameError: name 'np' is not defined

In [4]:
# parts

In [5]:
# Read part matrix

mapped_array = np.array([0,1,2,3,4],dtype = np.uint16)
mdir = np.zeros(maxorder,dtype = np.uint16)
mexp = np.zeros(maxorder,dtype = np.uint8)
print(mdir)
print(" -- -- ")
#get all partition sets
for i in range(A.shape[0]):        # Look up partition i
    print(" ----------- ----------- ----------- ")
    for j in range(A.shape[1]):    # Subpart j
        seti = A[i,j]
        part = []
        el = 0
        
        while seti != 0:
            
            if seti & 1:
                part.append(el)
            seti = (seti>>1)
            el += 1
        
        
        print(part)
        if len(part)>0:
            print(parts[i][j]==part)


[0 1 2 3 4]
 -- -- 
 ----------- ----------- ----------- 
[0, 1, 2, 3, 4]
True
[]
[]
[]
[]
 ----------- ----------- ----------- 
[0, 1, 2, 3]
True
[4]
True
[]
[]
[]
 ----------- ----------- ----------- 
[0, 1, 2, 4]
True
[3]
True
[]
[]
[]
 ----------- ----------- ----------- 
[0, 1, 2]
True
[3, 4]
True
[]
[]
[]
 ----------- ----------- ----------- 
[0, 1, 2]
True
[3]
True
[4]
True
[]
[]
 ----------- ----------- ----------- 
[0, 1, 3, 4]
True
[2]
True
[]
[]
[]
 ----------- ----------- ----------- 
[0, 1, 3]
True
[2, 4]
True
[]
[]
[]
 ----------- ----------- ----------- 
[0, 1, 3]
True
[2]
True
[4]
True
[]
[]
 ----------- ----------- ----------- 
[0, 1, 4]
True
[2, 3]
True
[]
[]
[]
 ----------- ----------- ----------- 
[0, 1]
True
[2, 3, 4]
True
[]
[]
[]
 ----------- ----------- ----------- 
[0, 1]
True
[2, 3]
True
[4]
True
[]
[]
 ----------- ----------- ----------- 
[0, 1, 4]
True
[2]
True
[3]
True
[]
[]
 ----------- ----------- ----------- 
[0, 1]
True
[2, 4]
True
[3]
True
[]
[]
 -----

In [19]:
mapped_array = np.array([1,2,3,4,5],dtype = np.uint16)
mdir = np.zeros(maxorder,dtype = np.uint16)
mexp = np.zeros(maxorder,dtype = np.uint8)
def expr():
    for i in range(A.shape[0]):        # Look up partition i
        print(" ----------- ----------- ----------- ")

        for j in range(A.shape[1]):    # Subpart j

            seti = A[i,j]
            
            if seti == 0:
                continue
            # end if
            print(seti)
            el = 0
            count = 0

            while seti != 0:

                if seti & 1:

                    mdir[count] = mapped_array[el]
                    count+=1

                # end if 

                seti = (seti>>1)
                el += 1

            # end while

            # fill up the other values as zeros:

            for k in range(count,mdir.size):

                mdir[k] = 0

            # end for
            print(mdir)

expr()            
#%timeit expr()
        


 ----------- ----------- ----------- 
31
[1 2 3 4 5]
 ----------- ----------- ----------- 
15
[1 2 3 4 0]
16
[5 0 0 0 0]
 ----------- ----------- ----------- 
23
[1 2 3 5 0]
8
[4 0 0 0 0]
 ----------- ----------- ----------- 
7
[1 2 3 0 0]
24
[4 5 0 0 0]
 ----------- ----------- ----------- 
7
[1 2 3 0 0]
8
[4 0 0 0 0]
16
[5 0 0 0 0]
 ----------- ----------- ----------- 
27
[1 2 4 5 0]
4
[3 0 0 0 0]
 ----------- ----------- ----------- 
11
[1 2 4 0 0]
20
[3 5 0 0 0]
 ----------- ----------- ----------- 
11
[1 2 4 0 0]
4
[3 0 0 0 0]
16
[5 0 0 0 0]
 ----------- ----------- ----------- 
19
[1 2 5 0 0]
12
[3 4 0 0 0]
 ----------- ----------- ----------- 
3
[1 2 0 0 0]
28
[3 4 5 0 0]
 ----------- ----------- ----------- 
3
[1 2 0 0 0]
12
[3 4 0 0 0]
16
[5 0 0 0 0]
 ----------- ----------- ----------- 
19
[1 2 5 0 0]
4
[3 0 0 0 0]
8
[4 0 0 0 0]
 ----------- ----------- ----------- 
3
[1 2 0 0 0]
20
[3 5 0 0 0]
8
[4 0 0 0 0]
 ----------- ----------- ----------- 
3
[1 2 0 0 0]
4
[3 0 0 0 0]
24

In [12]:
% timeit partition.Partition(mapped_array)

The slowest run took 5.06 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 14.7 µs per loop


In [22]:
bin(23)

'0b10111'